In [1]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import numpy as np
from backtesting import Strategy
from backtesting import Backtest
from sklearn.model_selection import ParameterGrid

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def getYFinanceData(ticker, period, interval):
  dataF = yf.download(ticker, period=period, interval=interval)

  dataF.iloc[:,:]
  #dataF.Open.iloc

  df = pd.DataFrame(dataF)

  # use df index, convert DateTime to  a column instead of index
  df.reset_index(inplace=True)

  # delete Adj Close
  df = df.drop(['Adj Close'], axis=1)

  # rename Datetime to "Gmt time"
  df = df.rename(columns={'Datetime':'Gmt time'})
  
  # rename Date to "Gmt time"
  df = df.rename(columns={'Date':'Gmt time'})

  df['Gmt time']=pd.to_datetime(df['Gmt time'],format='%d.%m.%Y %H:%M:%S')
  df.set_index("Gmt time", inplace=True)
  df=df[df.High!=df.Low]

  return df

In [3]:
def calculateTechnicalAnalysis(df):
  df["MACD"]=ta.macd(df.Close)['MACD_12_26_9']
  df["MACD_HIST"]=ta.macd(df.Close)['MACDh_12_26_9']
  df["MACD_SIGNAL"]=ta.macd(df.Close)['MACDs_12_26_9']
  df['RSI']=ta.rsi(df.Close, length=16)
  return df

In [4]:
def getDataPair(ticker):
  df = getYFinanceData(ticker, "365d", "60m")
  df1d = getYFinanceData(ticker, "365d", "1d")
  
  df = calculateTechnicalAnalysis(df)
  df1d = calculateTechnicalAnalysis(df1d)
  
  # create the column called Date. its value is the index without the time
  df['Date'] = df.index.date
  df1d['Date'] = df1d.index.date
  
  # Ensure 'Date' in df and index in df1d are in the same datetime format
  df['Date'] = pd.to_datetime(df['Date'])
  df1d.index = pd.to_datetime(df1d.index)

  # Use a try/except block to handle missing dates
  def get_macd(row):
      try:
          return df1d.loc[row['Date']]['MACD']
      except KeyError:
          return np.nan
      
      
  def get_macd_hist(row):
      try:
          return df1d.loc[row['Date']]['MACD_HIST']
      except KeyError:
          return np.nan
      
  def get_macd_signal(row):
      try:
          return df1d.loc[row['Date']]['MACD_SIGNAL']
      except KeyError:
          return np.nan
      
  def get_adx(row):
      try:
          return df1d.loc[row['Date']]['ADX']
      except KeyError:
          return np.nan
      


  df['MACD_1d'] = df.apply(get_macd, axis=1)
  df['MACD_HIST_1d'] = df.apply(get_macd_hist, axis=1)
  df['MACD_SIGNAL_1d'] = df.apply(get_macd_signal, axis=1)
  df['ADX_1d'] = df.apply(get_adx, axis=1)
  return df

MACD Signal

In [5]:
def getMACDSignals(df):
  backcandles = 15
  # calcualte macd signal
  macd_signal = [0]*len(df)
  for row in range(1, len(df)):
      if(
          True
          and df.MACD_HIST[row-1] < 0 and df.MACD_HIST[row] > 0
      ):
          macd_signal[row]=1
      elif  (
          True
          and df.MACD_HIST[row-1] > 0 and df.MACD_HIST[row] < 0
      ):
          macd_signal[row]=-1

  df['MACDSignal'] = macd_signal
  return df

RSI Signal

In [6]:
def getRSISignals(df):
  backcandles = 14

  # calcualte macd signal
  rsi_signal = [0]*len(df)
  df.RSI_14MA = df.RSI
  for row in range(1, len(df)):
      if row < backcandles:
          continue

      if(
          True
          and df.RSI[row] > 50
          or (df.RSI[row]<25 and df.RSI[row-1]>25)

      ):
          rsi_signal[row]=1
      elif  (
          True
          and df.RSI[row] < 55
          or (df.RSI[row]>75 and df.RSI[row-1]<75)
      ):
          rsi_signal[row]=-1

  df['RSI_Signal'] = rsi_signal
  return df

Total Signal

In [7]:
def TotalSignal(df, l):
  if (
      df.MACDSignal[l]==1
      and df.RSI_Signal[l]==1
  ):
          return 2
  if (
      df.MACDSignal[l]==-1
      and df.RSI_Signal[l]==-1
  ):
          return 1
  return 0
      
def assignTotalSignal(df):
  backcandles= 14
  TotSignal = [0]*len(df)
  for row in range(backcandles, len(df)): #careful backcandles used previous cell
      TotSignal[row] = TotalSignal(df, row)
  df['TotalSignal'] = TotSignal
  return df

In [8]:
yfinance_dict = {}

def backtest(ticker, atr_length, init_size, tp_sl_ratio, slatr_coef):
    df = None
    if (ticker in yfinance_dict):
        df = yfinance_dict[ticker]
    else :
        df = getDataPair(ticker)
        df = getMACDSignals(df)
        df = getRSISignals(df)
        df = assignTotalSignal(df)
        yfinance_dict[ticker] = df
    
    dfpl = df.copy()
    import pandas_ta as ta
    dfpl['ATR']=ta.atr(dfpl.High, dfpl.Low, dfpl.Close, length=atr_length)
    #help(ta.atr)
    def SIGNAL():
        return dfpl.TotalSignal

    class MyStrat(Strategy):
        initsize = init_size
        mysize = initsize
        latestEntry = 0
        lastHigh = 0
        
        def init(self):
            super().init()
            self.signal1 = self.I(SIGNAL)

        def next(self):
            super().next()
            slatr = slatr_coef*self.data.ATR[-1]
            TPSLRatio = tp_sl_ratio
            
            # if the current equity is higher than lastHigh, then update lastHigh
            if (self.equity > self.lastHigh):
                self.lastHigh = self.equity
                
            # if self.position.pl is down for more than 2 percent of the height, close all positions
            if (self.position.pl < -1 * self.lastHigh * 0.02):
                self.position.close()
                # print("Close position due to loss")
                # print("------")
            
            # # close the position if the amount lost exceeds 3% of the account balance
            if (self.position.pl < -1 * self.equity * 0.02):
                self.position.close()
                # print("Close position due to loss")
                # print("------")

            if len(self.trades)>0:
                for trade in self.trades:
                    if trade.is_long and self.data.RSI[-1]>=90:
                        trade.close()
                    elif trade.is_short and self.data.RSI[-1]<=10:
                        trade.close()
                    
                    # update the stop loss to entry if the pl is greater than the stop loss amount
                    # if (trade.pl > self.equity * 0.01):
                    #     trade.sl = trade.entry_price
                    #     print("Update stop loss to entry")
                    #     print("------")
                    
                    # close the position if the amount lost exceeds 3% of the account balance
                    # if (trade.pl < -1 * self.equity * 0.01):
                    #     trade.close()
                    #     print("Close position due to loss")
                    #     print("------")
                    
            # # # if the pl is greater than the stop loss amount, then move the stop loss to entry
            # if (self.position.pl > self.latestEntry * 0.02):
            #     # adjust the stop loss to entry
            #     self.position.sl = self.latestEntry
            #     # # for each opened position, update the stop loss to the entry
            #     # for trade in self.trades:
            #     #     trade.sl = self.latestEntry
                    
                    
            
            # print("Datetime: ", self.data.index[-1])
            if self.signal1==2 and len(self.trades) < 3:
                sl1 = self.data.Close[-1] - slatr
                tp1 = self.data.Close[-1] + slatr*TPSLRatio
                self.buy(sl=sl1, tp=tp1, size=self.mysize)
                self.latestEntry = self.data.Close[-1]
                # print("Buy", self.data.Close[-1])
                # print("Stop loss: ", sl1)
                # print("Take profit: ", tp1)
                # print("------")
            
            elif self.signal1==1 and len(self.trades) < 1:         
                sl1 = self.data.Close[-1] + slatr
                tp1 = self.data.Close[-1] - slatr*TPSLRatio
                self.sell(sl=sl1, tp=tp1, size=self.mysize)
                self.latestEntry = self.data.Close[-1]
                # print("Sell", self.data.Close[-1])
                # print("Stop loss: ", sl1)
                # print("Take profit: ", tp1)
                # print("------")

            # else:
            #     print("No signal")

    bt = Backtest(dfpl, MyStrat, cash=100000, margin=1/100, commission=0)
    stat = bt.run()
    return stat, bt

---

## Optimize Draw Down

In [9]:
# use gridsearchcv to find the best parameters

# turn off printing
import sys
sys.stdout = open('output.txt', 'w')

# Define your custom estimator and scoring method
class MyCustomEstimator:
    def __init__(self, ticker, atr_length, init_size, tp_sl_ratio, slatr_coef):
        self.ticker = ticker
        self.atr_length = atr_length
        self.init_size = init_size
        self.tp_sl_ratio = tp_sl_ratio
        self.slatr_coef = slatr_coef

    def score(self):
        # Your scoring logic here
        stat, bt = backtest(self.ticker, atr_length=self.atr_length, init_size=self.init_size , tp_sl_ratio=self.tp_sl_ratio, slatr_coef=self.slatr_coef)
        return stat["Max. Drawdown [%]"]

# Create a grid of hyperparameters
param_grid = {
    'ticker': ["^NDX", "^SPX", "AAPL"],
    'atr_length': [1, 4, 8, 12, 16],
    'init_size': [0.02, 0.03],
    'tp_sl_ratio': [1.5, 2],
    'slatr_coef': [4, 9, 12],
}

parameters_df= []

# Iterate over all combinations
for params in ParameterGrid(param_grid):
    estimator = MyCustomEstimator(**params)
    score = estimator.score()
    # Compare scores and find the best combination
    parameters_df.append([params, score])

parameters_df

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_88650/3184709326.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  and df.MACD_HIST[row-1] < 0 and df.MACD_HIST[row] > 0
/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_88650/3184709326.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  and df.MACD_HIST[row-1] > 0 and df.MACD_HIST[row] < 0
/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_88650/3003789530.py:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.RSI_14MA 

[[{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': '^NDX',
   'tp_sl_ratio': 1.5},
  -38.929788548052954],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': '^NDX',
   'tp_sl_ratio': 2},
  -32.716296612984785],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': '^SPX',
   'tp_sl_ratio': 1.5},
  -17.89484254714707],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': '^SPX',
   'tp_sl_ratio': 2},
  -25.540109760531436],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': 'AAPL',
   'tp_sl_ratio': 1.5},
  -28.479472760092484],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': 'AAPL',
   'tp_sl_ratio': 2},
  -36.56849359947899],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 9,
   'ticker': '^NDX',
   'tp_sl_ratio': 1.5},
  -26.878699480364897],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 9,
   'ticker': '^NDX',
   'tp_s

In [10]:
# find thee param tha resulted in the highest score
# max(parameters_df, key=lambda x: x[1])

ticker_groups = {}
for item in parameters_df:
    ticker = item[0]['ticker']
    if ticker not in ticker_groups:
        ticker_groups[ticker] = []
    ticker_groups[ticker].append(item)

# Find the item with the highest value in index 1 for each ticker
result = {}
for ticker, items in ticker_groups.items():
    max_item = max(items, key=lambda x: x[1])
    result[ticker] = max_item

result

{'^NDX': [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 9,
   'ticker': '^NDX',
   'tp_sl_ratio': 1.5},
  -26.878699480364897],
 '^SPX': [{'atr_length': 8,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': '^SPX',
   'tp_sl_ratio': 1.5},
  -13.936535977320696],
 'AAPL': [{'atr_length': 8,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': 'AAPL',
   'tp_sl_ratio': 2},
  -18.48934780641348]}

In [11]:
stat, bt = backtest("^SPX", atr_length=8, init_size=0.03 , tp_sl_ratio=1.5, slatr_coef=4)
bt.plot()
stat

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_88650/69321050.py:2: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot()


Start                     2022-11-23 09:30:00
End                       2024-05-08 13:30:00
Duration                    532 days 04:00:00
Exposure Time [%]                   67.007471
Equity Final [$]                252283.981764
Equity Peak [$]                 293287.182223
Return [%]                         152.283982
Buy & Hold Return [%]               28.807326
Return (Ann.) [%]                   89.438845
Volatility (Ann.) [%]               82.223989
Sharpe Ratio                         1.087746
Sortino Ratio                        3.718923
Calmar Ratio                         4.499442
Max. Drawdown [%]                  -19.877762
Avg. Drawdown [%]                   -3.913169
Max. Drawdown Duration       73 days 01:00:00
Avg. Drawdown Duration        8 days 14:00:00
# Trades                                   85
Win Rate [%]                        36.470588
Best Trade [%]                       3.994052
Worst Trade [%]                     -1.706328
Avg. Trade [%]                    

In [12]:
stat, bt = backtest("^NDX", atr_length=1, init_size=0.02 , tp_sl_ratio=2, slatr_coef=4)
bt.plot()
stat

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_88650/1630965305.py:2: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot()


Start                     2022-11-23 09:30:00
End                       2024-05-08 13:30:00
Duration                    532 days 04:00:00
Exposure Time [%]                    73.20442
Equity Final [$]                 193274.69043
Equity Peak [$]                 273777.274414
Return [%]                           93.27469
Buy & Hold Return [%]               52.727636
Return (Ann.) [%]                    57.80514
Volatility (Ann.) [%]               73.798519
Sharpe Ratio                         0.783283
Sortino Ratio                        2.175197
Calmar Ratio                         1.766861
Max. Drawdown [%]                  -32.716297
Avg. Drawdown [%]                   -4.134473
Max. Drawdown Duration      208 days 00:00:00
Avg. Drawdown Duration       11 days 03:00:00
# Trades                                   83
Win Rate [%]                         32.53012
Best Trade [%]                      18.040549
Worst Trade [%]                     -4.895707
Avg. Trade [%]                    

## Optimize Win Rate

In [13]:
# use gridsearchcv to find the best parameters

# turn off printing
import sys
sys.stdout = open('output.txt', 'w')

# Define your custom estimator and scoring method
class MyCustomEstimator:
    def __init__(self, ticker, atr_length, init_size, tp_sl_ratio, slatr_coef):
        self.ticker = ticker
        self.atr_length = atr_length
        self.init_size = init_size
        self.tp_sl_ratio = tp_sl_ratio
        self.slatr_coef = slatr_coef

    def score(self):
        # Your scoring logic here
        stat, bt = backtest(self.ticker, atr_length=self.atr_length, init_size=self.init_size , tp_sl_ratio=self.tp_sl_ratio, slatr_coef=self.slatr_coef)
        return stat["Win Rate [%]"]

# Create a grid of hyperparameters
param_grid = {
    'ticker': ["^NDX", "^SPX", "AAPL"],
    'atr_length': [1, 4, 8, 12, 16],
    'init_size': [0.02, 0.03],
    'tp_sl_ratio': [1.5, 2],
    'slatr_coef': [4, 9, 12],
}

parameters_df= []

# Iterate over all combinations
for params in ParameterGrid(param_grid):
    estimator = MyCustomEstimator(**params)
    score = estimator.score()
    # Compare scores and find the best combination
    parameters_df.append([params, score])

parameters_df

[[{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': '^NDX',
   'tp_sl_ratio': 1.5},
  32.22222222222222],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': '^NDX',
   'tp_sl_ratio': 2},
  32.53012048192771],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': '^SPX',
   'tp_sl_ratio': 1.5},
  50.0],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': '^SPX',
   'tp_sl_ratio': 2},
  40.67796610169492],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': 'AAPL',
   'tp_sl_ratio': 1.5},
  52.17391304347826],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': 'AAPL',
   'tp_sl_ratio': 2},
  46.875],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 9,
   'ticker': '^NDX',
   'tp_sl_ratio': 1.5},
  54.347826086956516],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 9,
   'ticker': '^NDX',
   'tp_sl_ratio': 2},
  51.85185185185185],

In [14]:
# find thee param tha resulted in the highest score
# max(parameters_df, key=lambda x: x[1])

ticker_groups = {}
for item in parameters_df:
    ticker = item[0]['ticker']
    if ticker not in ticker_groups:
        ticker_groups[ticker] = []
    ticker_groups[ticker].append(item)

# Find the item with the highest value in index 1 for each ticker
result = {}
for ticker, items in ticker_groups.items():
    max_item = max(items, key=lambda x: x[1])
    result[ticker] = max_item

result

{'^NDX': [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 12,
   'ticker': '^NDX',
   'tp_sl_ratio': 1.5},
  60.0],
 '^SPX': [{'atr_length': 8,
   'init_size': 0.02,
   'slatr_coef': 9,
   'ticker': '^SPX',
   'tp_sl_ratio': 1.5},
  58.536585365853654],
 'AAPL': [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': 'AAPL',
   'tp_sl_ratio': 1.5},
  52.17391304347826]}

In [20]:
stat, bt = backtest("^SPX", atr_length=12, init_size=0.02 , tp_sl_ratio=1.5, slatr_coef=9)
bt.plot()
stat

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_88650/805540974.py:2: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot()


Start                     2022-11-23 09:30:00
End                       2024-05-08 13:30:00
Duration                    532 days 04:00:00
Exposure Time [%]                   89.461266
Equity Final [$]                341768.034959
Equity Peak [$]                 397447.982713
Return [%]                         241.768035
Buy & Hold Return [%]               28.807326
Return (Ann.) [%]                  133.611402
Volatility (Ann.) [%]              111.724172
Sharpe Ratio                         1.195904
Sortino Ratio                        4.818783
Calmar Ratio                         6.511684
Max. Drawdown [%]                  -20.518717
Avg. Drawdown [%]                   -3.067073
Max. Drawdown Duration      127 days 00:00:00
Avg. Drawdown Duration        6 days 00:00:00
# Trades                                   41
Win Rate [%]                        58.536585
Best Trade [%]                       9.191412
Worst Trade [%]                      -3.24026
Avg. Trade [%]                    

In [16]:
stat, bt = backtest("^NDX", atr_length=1, init_size=0.02 , tp_sl_ratio=1.5, slatr_coef=9)
bt.plot()
stat

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_88650/2712909604.py:2: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot()


Start                     2022-11-23 09:30:00
End                       2024-05-08 13:30:00
Duration                    532 days 04:00:00
Exposure Time [%]                   82.675612
Equity Final [$]                481798.225098
Equity Peak [$]                 524078.977051
Return [%]                         381.798225
Buy & Hold Return [%]               52.727636
Return (Ann.) [%]                  196.997523
Volatility (Ann.) [%]              189.903122
Sharpe Ratio                         1.037358
Sortino Ratio                        5.731842
Calmar Ratio                         7.329132
Max. Drawdown [%]                  -26.878699
Avg. Drawdown [%]                   -3.745192
Max. Drawdown Duration      146 days 23:00:00
Avg. Drawdown Duration        6 days 07:00:00
# Trades                                   46
Win Rate [%]                        54.347826
Best Trade [%]                      16.004109
Worst Trade [%]                     -5.017563
Avg. Trade [%]                    

## Higheest return

In [17]:
stat2, bt2 = backtest("^NDX", atr_length=1, init_size=0.02 , tp_sl_ratio=2, slatr_coef=4)
bt.plot(show_legend=False)
stat2

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_88650/41808923.py:2: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot(show_legend=False)


Start                     2022-11-23 09:30:00
End                       2024-05-08 13:30:00
Duration                    532 days 04:00:00
Exposure Time [%]                    73.20442
Equity Final [$]                 193274.69043
Equity Peak [$]                 273777.274414
Return [%]                           93.27469
Buy & Hold Return [%]               52.727636
Return (Ann.) [%]                    57.80514
Volatility (Ann.) [%]               73.798519
Sharpe Ratio                         0.783283
Sortino Ratio                        2.175197
Calmar Ratio                         1.766861
Max. Drawdown [%]                  -32.716297
Avg. Drawdown [%]                   -4.134473
Max. Drawdown Duration      208 days 00:00:00
Avg. Drawdown Duration       11 days 03:00:00
# Trades                                   83
Win Rate [%]                         32.53012
Best Trade [%]                      18.040549
Worst Trade [%]                     -4.895707
Avg. Trade [%]                    

In [18]:
stat, bt = backtest("^NDX", atr_length=1, init_size=0.03 , tp_sl_ratio=1.5, slatr_coef=9)
bt.plot()
stat


/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_88650/519562990.py:2: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot()


Start                     2022-11-23 09:30:00
End                       2024-05-08 13:30:00
Duration                    532 days 04:00:00
Exposure Time [%]                    81.33386
Equity Final [$]                569913.904297
Equity Peak [$]                 641765.822266
Return [%]                         469.913904
Buy & Hold Return [%]               52.727636
Return (Ann.) [%]                   233.62037
Volatility (Ann.) [%]              344.550414
Sharpe Ratio                         0.678044
Sortino Ratio                        4.693674
Calmar Ratio                         6.441276
Max. Drawdown [%]                  -36.269271
Avg. Drawdown [%]                   -7.424607
Max. Drawdown Duration      153 days 03:00:00
Avg. Drawdown Duration        9 days 05:00:00
# Trades                                   55
Win Rate [%]                        43.636364
Best Trade [%]                      16.004109
Worst Trade [%]                     -3.604716
Avg. Trade [%]                    

In [19]:
stat, bt = backtest("BTC-USD", atr_length=1, init_size=0.03 , tp_sl_ratio=1.5, slatr_coef=1)
bt.plot()
stat


/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_88650/3184709326.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  and df.MACD_HIST[row-1] < 0 and df.MACD_HIST[row] > 0
/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_88650/3184709326.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  and df.MACD_HIST[row-1] > 0 and df.MACD_HIST[row] < 0
/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_88650/3003789530.py:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.RSI_14MA 

Start                     2023-05-10 00:00:00
End                       2024-05-08 18:00:00
Duration                    364 days 18:00:00
Exposure Time [%]                   28.206596
Equity Final [$]                 92816.021484
Equity Peak [$]                 127629.650391
Return [%]                          -7.183979
Buy & Hold Return [%]              125.435092
Return (Ann.) [%]                   -7.183979
Volatility (Ann.) [%]               53.662119
Sharpe Ratio                        -0.133874
Sortino Ratio                       -0.204023
Calmar Ratio                        -0.217594
Max. Drawdown [%]                  -33.015539
Avg. Drawdown [%]                  -10.246121
Max. Drawdown Duration      163 days 10:00:00
Avg. Drawdown Duration       27 days 00:00:00
# Trades                                  368
Win Rate [%]                        35.597826
Best Trade [%]                       4.232624
Worst Trade [%]                     -2.129031
Avg. Trade [%]                    